# AutoKeras 

- pip install autokeras
- https://github.com/jhfjhfj1/autokeras
- https://arxiv.org/abs/1806.10282

In [2]:
### Tensorflow : 1.10 

from keras.datasets import mnist
from autokeras import ImageClassifier
import tensorflow

/Users/dhlee/anaconda/envs/tensorflow3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## Load MNIST data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

## Train

In [4]:
clf = ImageClassifier(verbose=True, searcher_args={'trainer_args':{'max_iter_num':5}})

In [6]:
clf.fit(x_train, y_train, time_limit=5 * 60 * 60)


Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   4.942196583747863    |        0.96212         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+


/home/dhlee-volta/anaconda3/lib/python3.6/site-packages/autokeras/bayesian.py:151: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "



+--------------------------------------------------------------------------+
|    Father Model ID     |                 Added Operation                 |
+--------------------------------------------------------------------------+
|           0            |          ('to_concat_skip_model', 1, 5)         |
+--------------------------------------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           1            |   3.2061469621956347   |         0.974          |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 2               |
+----------------------------------------------+

+---------------------------------------------------------------



+----------------------------------------------+
|               Training model 7               |
+----------------------------------------------+

+--------------------------------------------------------------------------+
|    Father Model ID     |                 Added Operation                 |
+--------------------------------------------------------------------------+
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 5, 3)         |
|                        |          ('to_conv_deeper_model', 9, 3)         |
|                        |           ('to_add_skip_model', 1, 18)          |
|                        |          ('to_add_skip_model', 18, 24)          |
|                        |          ('to_add_skip_model', 24, 27)          |
|           5            |         ('to_conv_deeper_model', 24, 3)         |
|  



+----------------------------------------------+
|              Training model 12               |
+----------------------------------------------+

+--------------------------------------------------------------------------+
|    Father Model ID     |                 Added Operation                 |
+--------------------------------------------------------------------------+
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 5, 3)         |
|                        |          ('to_conv_deeper_model', 9, 3)         |
|                        |           ('to_add_skip_model', 1, 18)          |
|                        |          ('to_add_skip_model', 18, 24)          |
|                        |          ('to_add_skip_model', 24, 27)          |
|           6            |         ('to_conv_deeper_model', 24, 3)         |
|  



+----------------------------------------------+
|              Training model 17               |
+----------------------------------------------+

+--------------------------------------------------------------------------+
|    Father Model ID     |                 Added Operation                 |
+--------------------------------------------------------------------------+
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 1, 3)         |
|                        |          ('to_conv_deeper_model', 5, 3)         |
|                        |          ('to_conv_deeper_model', 9, 3)         |
|                        |           ('to_add_skip_model', 1, 18)          |
|                        |          ('to_add_skip_model', 18, 24)          |
|                        |          ('to_add_skip_model', 24, 27)          |
|                        |         ('to_conv_deeper_model', 24, 3)         |
|  



+----------------------------------------------+
|              Training model 21               |
+----------------------------------------------+

Time limit for model search is reached. Ending the model search.


## Train the best model

In [10]:
clf.final_fit(x_train, y_train, x_test, y_test, retrain=False, trainer_args={'max_iter_num':10})


Loading and training the best model recorded from the search.


## Test

In [13]:
y = clf.evaluate(x_test, y_test)
print(y * 100)

99.13


## Best Model Architecture Overview

In [22]:
best_model = clf.load_searcher().load_best_model()

In [15]:
best_model.n_layers

67

In [16]:
from torchvision import models
print(best_model.produce_model())

TorchModel(
  (0): ReLU()
  (1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): ReLU()
  (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): ReLU()
  (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): TorchFlatten()
  (13): Dropout2d(p=0.25)
  (14): Linear(in_features=576, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=10, bias=True)
  (17): ReLU()
  (18): Conv2d(128, 6

## Save Model

In [27]:
import torch

torch.save(best_model, 'autoKeras_mnist.h5')
model = torch.load('autoKeras_mnist.h5')
print(model.produce_model())


TorchModel(
  (0): ReLU()
  (1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): ReLU()
  (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): ReLU()
  (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): TorchFlatten()
  (13): Dropout2d(p=0.25)
  (14): Linear(in_features=576, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=10, bias=True)
  (17): ReLU()
  (18): Conv2d(128, 6